In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
rootfolder = ".."

Load the image and rescale it in $[0,1]$


In [ ]:
img = imread(f"{rootfolder}/data/cameraman.png") / 255
img = img[50:200, 50:200]
imsz = img.shape

# patch size
p = 7

# number of elements in the patch
M = p**2

# radius of the search neighborhood
H = 12

Corrupt the image with white gaussian noise


In [ ]:
sigma_noise = 20 / 255
noisy_img = img + np.random.normal(size=imsz) * sigma_noise

Compute the PSNR of the noisy input


In [ ]:
psnr_noisy = 10 * np.log10(1 / np.mean((noisy_img - img) ** 2))
psnr_noisy

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img, cmap="gray")
ax[0].set_title("Original image")

ax[1].imshow(noisy_img, cmap="gray")
ax[1].set_title(f"Noisy image, PSNR = {psnr_noisy:.2f}")

## Pixel-based NL-Means

Set parameters and initialize the variables


In [ ]:
# initialize the estimated image
img_hat = np.zeros_like(img)

# pad the noisy image
img_pad = np.pad(
    noisy_img, ((H + p // 2, H + p // 2), (H + p // 2, H + p // 2)), mode="reflect"
)

Main loop


In [ ]:
# filtering parameter
h = 0.1
for i in range(0, imsz[0]):
    for j in range(0, imsz[1]):
        # extract the patch from img_pad whose center has the location (i, j) in the noisy image
        s = img_pad[i + H : i + H + p, j + H : j + H + p]

        # initialize the pixel estimate and the total weights
        pixel_hat = 0
        weight = 0

        # extract all the patches in the search neighborhood
        for r in range(max(0, i - H), min(imsz[0], i + H + 1)):
            for c in range(max(0, j - H), min(imsz[1], j + H + 1)):
                # extract the patch
                z = img_pad[r + H : r + H + p, c + H : c + H + p]

                # compute the distance with the reference patch
                d = np.sum((s - z) ** 2) / M

                # compute the weight
                w = np.exp(-d / (h**2))

                # update the weight and the pixel estimate
                pixel_hat += w * noisy_img[r, c]
                weight += w

        # estimate the pixel (i, j) as the weighted average of the central pixel of the extracted patches
        img_hat[i, j] = pixel_hat / weight

Compute the PSNR of the estimated image


In [ ]:
psnr_hat = 10 * np.log10(1 / np.mean((img_hat - img) ** 2))
plt.figure(figsize=(10, 10))
plt.imshow(img_hat, cmap="gray")
plt.title(f"Estimated Image,\nPSNR = {psnr_hat:.2f}")


